# install packages

In [7]:
!pip install emoji nagisa 
!pip install -U ginza ja-ginza ja_ginza
!python3 -m spacy download ja_ginza
!python3 -m spacy download ja_core_news_sm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
/home/nb-user/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-05-19 15:07:22.441646: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-19 15:07:25.813196: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

✘ No compatible package found for 'ja_ginza' (spaCy v3.4.4)

/home/nb-user/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initia

# Load Dataset

In [6]:
import json
import pandas as pd

danwasMap = {'並列':0, '対比':1, '要望':2, '条件':3, '原因結果':4, '例提示':5, '詳細化':6, '主題連鎖':7, '焦点主題連鎖':8,'順接':9, '逆接':10, '短文':11, '質問応答':12, '理由':13}
labelsMap = {0: "感情的攻撃", 1: "感情的説得", 2: "理性的説得", 3: "嫌味皮肉", 4: "遠回し", 5: "怒っていない"}

baseData = pd.read_json("/home/nb-user/ksl_pub/suggestive_anger/data/fuman_survey/json/human_data_ver5.json")
baseData = baseData.rename(columns={'fulltext': 'text', "label": "labels"})
baseData['danwa_result'] = baseData['danwa_result'].map(danwasMap).fillna(3).astype('int64')
# baseData = baseData[["text", "labels"]]
# baseData.to_csv("/content/drive/MyDrive/LAB/研究/human_data_ver5.csv")

kushiroFeature = pd.read_csv("/home/nb-user/ksl_pub/suggestive_anger/data/fuman_survey/kosen_features.csv")
kushiroLabel = pd.read_csv("/home/nb-user/ksl_pub/suggestive_anger/data/fuman_survey/kosen_labels.csv")
kushiroData = pd.concat([kushiroFeature, kushiroLabel], axis=1)
kushiroData = kushiroData.rename(columns={'feature': 'text', "label": "labels"})

baseData = pd.concat([baseData, kushiroData])
baseData = baseData.reset_index(
    drop=True # オリジナルのindexを列として保存しない
)

print(baseData.labels.value_counts())
baseData


labels
2    1076
1     758
0     282
4     233
5     227
3     224
Name: count, dtype: int64


,text,sentences,danwa_result,result_form,labels
0,最近の内容は、のび太が、他力本願ですぐ調子にのり、エゴが強すぎると感じている。子供に見せたく...,"[最近の内容は、のび太が、他力本願ですぐ調子にのり、エゴが強すぎると感じている, 子供に見せ...",4.0,"{'3': '1.0', '0': '1.0', '1': '7.0', '2': '4.0'}",1
1,ヤクルトの舌や喉に何か残る感じの、あれは何でしょう?すごくいやだ。,[ヤクルトの舌や喉に何か残る感じの、あれは何でしょう?すごくいやだ],6.0,"{'0': '7.0', '1': '2.0', '3': '2.0', '2': '1.0...",0
2,めばちこ痛い。治ってもまたできる...,"[めばちこ痛い, 治ってもまたできる...]",0.0,"{'0': '1.0', '5': '12.0'}",5
3,タケダは、からだ浸透補水液は病院の消毒液を連想させる味がします。体に良くても飲み物としてなじ...,"[タケダは、からだ浸透補水液は病院の消毒液を連想させる味がします, 体に良くても飲み物として...",5.0,"{'2': '1.0', '1': '3.0', '4': '1.0', '3': '8.0'}",3
4,チョコリングの個性が弱くて商品として地味で美味しくない。,[チョコリングの個性が弱くて商品として地味で美味しくない],9.0,"{'0': '1.0', '2': '8.0', '5': '1.0', '1': '3.0'}",2
...,...,...,...,...,...
2795,マクドナルドなのにポケモンgoのストップが効かない。ストップはマックドナルドの表示になってる...,NaN,NaN,NaN,2
2796,早くポケモンを追加して欲しい。そろそろ飽きてきます,NaN,NaN,NaN,2
2797,ゲームセンターCXを深夜枠でもいいから地上波でしてもらいたい。,NaN,NaN,NaN,2
2798,ヤフーニュースのコメントが酷いの多くて不満。誹謗中傷とかは削除するようにしてほしいわ。,NaN,NaN,NaN,1


# Load functions

In [8]:
import emoji
import nagisa
import unicodedata
import pandas as pd


def extract_emoji(words):
    """
    日本語のテキストから絵文字😉を抽出する  
    （顔文字は抽出しない）
    """
    words = str(words)
    return [w for w in words if emoji.is_emoji(str(w))]
    # return [w for w in words if w in emoji.UNICODE_EMOJI_ENGLISH]


# text = "今日は渋谷スクランブルスクエアに行ってきた＼(^o^)／ 夜景🏙サイコー❗️ https://hogehogehogehoge.jpg"
# text = unicodedata.normalize('NFKC', text) # NFKC正規化
# print(text)
# print(extract_emoji(text))
# # => ['🏙', '❗']

# text = "日本語のテキストから絵文字😉を抽出するよ❗"
# text = unicodedata.normalize('NFKC', text) # NFKC正規化
# print(text)
# print(extract_emoji(text))
# # => ['😉', '❗']


# for i, data in enumerate(baseData.values):
#     feature = data[0]
#     if i < 10:
#         print(f'i : {i}\tfeature : {feature}')
#     if '(*́ー`*)' in feature:
#         print(i)

[dynet] random seed: 1234
[dynet] allocating memory: 32MB
[dynet] memory allocation done.


In [9]:
KAOMOJI_LEN = 5

def extract_kaomoji(text):
    """ 与えられたテキストから抽出した顔文字リストを返却する。
        → ＼(^o^)／, m(_ _)m などの 手を含む顔文字があれば、それも抽出する
    """
    results = nagisa.extract(text, extract_postags=['補助記号'])
    words = results.words
    kaomoji_words = []
    kaomoji_idx = [i for i, w in enumerate(words) if len(w) >= KAOMOJI_LEN]
    kaomoji_hands = ['ノ', 'ヽ', '∑', 'm', 'O', 'o', '┐', '/', '\\', '┌'] 
    # 顔文字と手を検索
    for i in kaomoji_idx:
        kaomoji = words[i] # 顔文字列
        try:
            # 顔文字の左手
            if words[i-1] in kaomoji_hands and 0 < i:
                kaomoji = words[i-1] + kaomoji
            # 顔文字の右手
            if words[i+1] in kaomoji_hands:
                kaomoji = kaomoji + words[i+1]
        except IndexError:
            pass
        finally:
            kaomoji_words.append(kaomoji)
    return kaomoji_words


# for index, row in baseData.iterrows():
#   text = unicodedata.normalize('NFKC', row["text"]) # NFKC正規化
#   emoji = extract_kaomoji(text)
#   if emoji: 
#     print(index, emoji)

In [10]:
import spacy

# nlp = spacy.load("ja_ginza")
nlp = spacy.load("ja_core_news_sm")

def separate_sentence(feature):
    doc = nlp(feature)
    return [str(sent) for sent in doc.sents]


def delete(text, target_list):
    for target in target_list:
        text = text.replace(target, '')
    return text


# for index, row in baseData.iterrows():
#   text = unicodedata.normalize('NFKC', row["text"].replace(' ', ''))
#   target_list = extract_kaomoji(text)
#   cleaned_text = delete(text, target_list)
#   if target_list:
#     [print(target_list)]
#     print(row["text"].replace(' ', ''))
#     print(separate_sentence(cleaned_text))



In [11]:
def replace_emoji(sentence):
    return sentence.replace('❗', '!').replace('❓', '?').replace('‼️', '!!').replace('⁉️', '!?')
    # return sentence.replace('❗', '！').replace('❓', '？').replace('‼️', '！！').replace('⁉️', '！？')


def delete_emoji(sentence):
    emoji_list = extract_emoji(sentence)
    if len(emoji_list) == 0:
        return sentence
    
    for e in emoji_list:
        sentence = sentence.replace(e, '')
    return sentence


def delete_kaomoji(sentence):
    sentence = sentence.replace('　', '').replace(' ', '')
    
    if '(^^;;' in sentence:
        sentence = sentence.replace('(^^;;', '')
    
    if '́д`;' in sentence:
        sentence = sentence.replace('́д`;', '')
    
    kaomoji_list = extract_kaomoji(sentence)
    if len(kaomoji_list) == 0:
        return sentence
    
    for kaomoji in kaomoji_list:
        """unicode_sentence = unicodedata.normalize('NFKC', sentence)
        print(kaomoji)
        print(unicode_sentence)
        print(kaomoji in unicode_sentence)"""
        sentence = sentence.replace(kaomoji, '')
    return sentence





In [12]:
# def check(position, sentences):
#     for i, sentence in enumerate(sentences):
#         # print(sentence)
#         if sentence[0] == '(':
#             print(sentence)
#             print('')
#             print(sentences)
#             print('')
#     return

import re
# pattern = '^[.,!?/、。・？！]+$'
# repatter = re.compile(pattern)
# def fix_symbol(sentences):
#     return [s for s in sentences if repatter.match(s) is None]

tilde_pattern = '[\u007E\u02DC\u0303\u1FC0\u2053\u223C\u223F\u3030\uFF5E]'
comma_pattern = r'[､、]{2,}'
full_stop_pattern = r'[｡。]{2,}'
unnecessary_full_stop_pattern = r'([!?]+)[｡。]+'
number_pattern = r'[0-9]+'
compiled_tilde = re.compile(tilde_pattern)
compiled_comma = re.compile(comma_pattern)
compiled_full_stop = re.compile(full_stop_pattern)
compiled_unnecessary = re.compile(unnecessary_full_stop_pattern)
compiled_number = re.compile(number_pattern)


def delete_slash(feature):
    return feature.replace('/', '')


def fix_tilde(feature):
    return compiled_tilde.sub('〜', feature)


def fix_multiple_punctuation(feature):
    return compiled_comma.sub(
        '、', compiled_full_stop.sub(
            '。', compiled_unnecessary.sub(r'\1', feature)
        )
    )


def delete_irregular_str(feature):
    return feature.replace('\uFE0E', '').replace('\uFE0F', '')


def replace_number_to_zero(feature):
    match_list = compiled_number.findall(feature)
    for match_str in match_list:
        feature = feature.replace(match_str, '0')
    return feature


# 文ごとに分けたあとの処理
def fix_bracket(sentences, left_bracket, right_bracket):
    count_left_bracket  = sum([s.count(left_bracket) for s in sentences])
    count_right_bracket = sum([s.count(right_bracket) for s in sentences])
    
    if (count_left_bracket == 0
        and count_left_bracket == count_right_bracket):
        return sentences
    
    # print(f'count_left_bracket : {count_left_bracket}')
    sentence_length = len(sentences)
    new_sentences = []
    should_skip = False
    
    for i in range(sentence_length):
        if should_skip:
            should_skip = False
            continue
        
        if sentences[i].count(left_bracket) == sentences[i].count(right_bracket):
            new_sentences.append(sentences[i])
            continue
        
        right_bracket_position = i + 1
        
        if (i != sentence_length - 1
            and sentences[right_bracket_position].count(left_bracket)
            != sentences[right_bracket_position].count(right_bracket)):
            new_sentences.append(sentences[i] + sentences[right_bracket_position])
            should_skip = True
    
    return new_sentences


symbol_pattern = r'^[.!?]+'
leader_pattern = r'[.・]{2,}'
end_of_sentence_pattern = r'[。!?]$'
compiled_symbol = re.compile(symbol_pattern)
compiled_leader = re.compile(leader_pattern)
compiled_eos = re.compile(end_of_sentence_pattern)


def join_symbol_only_sentence(sentences):
    sentence_length = len(sentences)
    new_sentences = []
    is_matched = False
    
    for i in range(sentence_length):
        if is_matched:
            is_matched = False
            continue
        
        if (compiled_symbol.match(sentences[i]) is None
            or i == 0):
            new_sentences.append(sentences[i])
            continue
        
        new_sentences[-1] += sentences[i]
        should_skip = True
    
    return new_sentences


def restore_leader(sentences):
    return [compiled_leader.sub('…', sentence)
            if compiled_leader.search(sentence)
            else sentence for sentence in sentences]


def delete_sentence_only_symbols(sentences):
    return [s for s in sentences if s != '、']


def join_sentence_if_start_left_paren(sentences):
    new_sentence = []
    for i, sentence in enumerate(sentences):
        if i == 0:
            new_sentence.append(sentence)
            continue
        
        if sentence[0] == '(':
            new_sentence[-1] += sentence
        else:
            new_sentence.append(sentence)
    
    return new_sentence


def fix_end_of_sentence(sentences):
    return [sentence if compiled_eos.search(sentence)
            else sentence + '。' for sentence in sentences]


def make_sentences_from_feature(feature):
    sentences = separate_sentence(
        replace_number_to_zero(
            delete_irregular_str(
                fix_multiple_punctuation(
                    fix_tilde(
                        delete_slash(
                            delete_emoji(
                                delete_kaomoji(
                                    replace_emoji(feature)
                                )
                            )
                        )
                    )
                )
            )
        )
    )
    return fix_end_of_sentence(
        join_sentence_if_start_left_paren(
            delete_sentence_only_symbols(
                restore_leader(
                    join_symbol_only_sentence(
                        fix_bracket(
                            fix_bracket(
                                fix_bracket(
                                    sentences, '「', '」'),
                                '『', '』'),
                            '(', ')')
                    )
                )
            )
        )
    )

In [13]:
features = []

for i, data in enumerate(baseData.values):
  feature = data[0]
  feature = unicodedata.normalize('NFKC', feature) # NFKC正規化
  # emoji_list = extract_emoji(feature)
  result = make_sentences_from_feature(feature)
  line_length = len(result)
  sentence = ' '.join(result)
  features.append([i, line_length, sentence]) # [index, 文章数, 本文]
  # for sentence_i, sentence in enumerate(result):
  #     # index, 何文章目か, 文章数, 本文
  #     features.append([i, sentence_i, len_result, sentence]) 


[[0, 2, '最近の内容は、のび太が、他力本願ですぐ調子にのり、エゴが強すぎると感じている。 子供に見せたくないアニメになった。'], [1, 2, 'ヤクルトの舌や喉に何か残る感じの、あれは何でしょう? すごくいやだ。'], [2, 2, 'めばちこ痛い。 治ってもまたできる…。'], [3, 2, 'タケダは、からだ浸透補水液は病院の消毒液を連想させる味がします。 体に良くても飲み物としてなじまない味ですねっ。'], [4, 1, 'チョコリングの個性が弱くて商品として地味で美味しくない。'], [5, 2, 'ペットボトルは両手を使わないと開けられないので不満。 片手ワンタッチで開けられるものがあればいいなぁ。'], [6, 2, '水道の蛇口のランプがなぜ赤色なんでしょうか。 水色が青色のランプにして欲しかった。'], [7, 1, '動画を見てポイントを貯めるのがあるが、動画によっては何度見てもポイントが反映されないものがある。'], [8, 2, 'わかるとすでにあるって意味ある? 間違って押してもキャンセルできないしべつになくていい。'], [9, 1, 'ディスカウントショップで非常口前に安売り商品が山積みして有り、もしもの時に危ない。'], [10, 1, '夜になると老婆が徘徊し始めるのは田舎だから?'], [11, 1, '一人で飲みに行ける居酒屋さん、開拓するまで勇気がいる。'], [12, 1, '冷房を付けて寝ると朝起きてから、くしゃみや鼻水が止まらなくなる事がある。'], [13, 1, '背中を日焼けして、皮が剥けて来ると、布団が剥けた皮で散らかる。'], [14, 1, 'チェキカメラ、内カメラでもとれるようにしてほしい。'], [15, 2, '猫ブームとやらで猫ばかり取り上げるのが不満。 猫嫌いな人もいるんだよ。'], [16, 1, 'ポケモンGOのせいで歩きスマホをする人が目に見えて増えた。'], [17, 1, '洗い物をすると手が乾燥してカサカサになってしまう。'], [18, 2, 'オリンピックの情報が多くて飽きる。 それほど興味があるわけではない。'], [19, 1, '最近の雑誌は高い中身の内容薄いのに千円近くするおまけもいらない。'], [20, 2, '水筒のパッキンが黒くなる。 親がちゃんと洗ってても

In [14]:
features

[[0, 2, '最近の内容は、のび太が、他力本願ですぐ調子にのり、エゴが強すぎると感じている。 子供に見せたくないアニメになった。'],
 [1, 2, 'ヤクルトの舌や喉に何か残る感じの、あれは何でしょう? すごくいやだ。'],
 [2, 2, 'めばちこ痛い。 治ってもまたできる…。'],
 [3, 2, 'タケダは、からだ浸透補水液は病院の消毒液を連想させる味がします。 体に良くても飲み物としてなじまない味ですねっ。'],
 [4, 1, 'チョコリングの個性が弱くて商品として地味で美味しくない。'],
 [5, 2, 'ペットボトルは両手を使わないと開けられないので不満。 片手ワンタッチで開けられるものがあればいいなぁ。'],
 [6, 2, '水道の蛇口のランプがなぜ赤色なんでしょうか。 水色が青色のランプにして欲しかった。'],
 [7, 1, '動画を見てポイントを貯めるのがあるが、動画によっては何度見てもポイントが反映されないものがある。'],
 [8, 2, 'わかるとすでにあるって意味ある? 間違って押してもキャンセルできないしべつになくていい。'],
 [9, 1, 'ディスカウントショップで非常口前に安売り商品が山積みして有り、もしもの時に危ない。'],
 [10, 1, '夜になると老婆が徘徊し始めるのは田舎だから?'],
 [11, 1, '一人で飲みに行ける居酒屋さん、開拓するまで勇気がいる。'],
 [12, 1, '冷房を付けて寝ると朝起きてから、くしゃみや鼻水が止まらなくなる事がある。'],
 [13, 1, '背中を日焼けして、皮が剥けて来ると、布団が剥けた皮で散らかる。'],
 [14, 1, 'チェキカメラ、内カメラでもとれるようにしてほしい。'],
 [15, 2, '猫ブームとやらで猫ばかり取り上げるのが不満。 猫嫌いな人もいるんだよ。'],
 [16, 1, 'ポケモンGOのせいで歩きスマホをする人が目に見えて増えた。'],
 [17, 1, '洗い物をすると手が乾燥してカサカサになってしまう。'],
 [18, 2, 'オリンピックの情報が多くて飽きる。 それほど興味があるわけではない。'],
 [19, 1, '最近の雑誌は高い中身の内容薄いのに千円近くするおまけもいらない。'],
 [20, 2, '水筒のパッ